# Key Methods: Week 10

# Spatial data analysis and batch processing in Python and GIS

In [ ]:
# run this to display graphs nicely in this notebook

%matplotlib inline


## Introduction

This week we will use a mixture of python and GIS to process a large set of data. GIS allows us to visually assess and interact with each step, whilst python allows us to easily scale up to processing large datasets with a single button press, making repeat processing easy. We will answer the same question as week 2; using lidar data to map biomass along the Spey river valley.

## Data

Again we are using the DTM and DSM provided by the [Scottish government](https://remotesensingdata.gov.scot/data#/list). These have been provided through Noteable in the ***data*** directory. There are three folders containing the DSM and DTMs at 10 m resolution, and one with a single DSM tile at 1 m resoluton.

## Image maps

File $\texttt{ArthursSeat.txt}$ contains elevations in metres on an 800×800 grid with 2 m horizontal spacing for our very own Carboniferous volcano. This is a text file containing the geospatial data. Open the file and have a look. Satisfy yourself that it is a 2D spreadsheet of elevation values. Later we will see some other common, and more efficient, spatial data formats.

Write a program to read the elevation file into an array `z`. Simply calling `plt.imshow(z)` will now make an image map, but there are several things that we will want to change about this default map.

First, note that the numbers on the y axis are in reverse order (and, if you are familiar with the topography of Arthur’s Seat, you will see that the map is inverted). This follows the convention for storing images (i.e. a digital photograph will be shown correctly) but can be corrected for map axes with an origin in the lower left corner by adding keyword argument `origin='lower'`. 

Keyword argument `cmap` in `plt.imshow()` selects the colour map. The default is called `'viridis'`, which is a perceptually uniform sequential colour map (one which gives uniform steps in perceived brightness for uniform steps in data values), but `cmap='gray'` gives a greyscale which works perfectly well for this example. Experiment with the matplotlib colour maps listed at  http://matplotlib.org/users/colormaps.html to find ones that you think are particularly good or bad for showing this data. Until recently, the matplotlib default was the rainbow scale called `'jet'`, and you will still see many examples of plots with rainbow scales in research literature, even though they are now considered to be a poor choice.

The axes are labelled in grid coordinates (numbers of points) rather than geographical coordinates. The correct extents of the axes (800 × 2 = 1600 m) can be set with keyword argument `extent=(0,1600,0,1600)`. Use `plt.xticks()` and `plt.yticks()` if you want to control where the axis tick marks are placed and how they are labelled.

To have a colour bar beside the map, add `cbar = plt.colorbar()`. 

Having assigned a name to the colour bar object, there are several methods that can modify it. Use the `set_label()` method with a text string to label the colour bar, e.g. `cbar.set_label('Elevation (m)')`, and use `cbar.set_ticks(list)` with a list of numbers to set the locations of tick marks on the bar.


## Gradients and slopes

The components of the gradient vector for array `z` can be estimated by central differences using 
```
dzdy, dzdx = np.gradient(z, dx)
```
where `dx` is the grid spacing (2 m for the Arthur’s Seat DEM). The slope is calculated and converted to slope angle in degrees by 
```
slope = np.sqrt(dzdx**2 + dzdy**2)
angle = np.rad2deg(np.arctan(slope))
```

Make a figure with three images of `dzdx`, `dzdy` and `angle` side by side. In this layout, you might think that horizontal colour bars under the plots will fit better; use `plt.colorbar(orientation='horizontal')` to do this. If you want to shrink or expand the bar to a fraction of its default size, use keyword argument `shrink`.

Because `dzdx` and `dzdy` have both positive and negative values, use a diverging scale from the list of colour maps. Ensure that zero is in the middle of the scale by setting symmetrical negative and positive values for keyword arguments `vmin` and `vmax` in `plt.imshow()`.


Find the maximum slope angle. The places with the steepest slopes should be clearly visible in the slope angle image; check that they are where you expect them to be.


## Topographic shading

A shaded surface map can be created with the lines
```
from matplotlib.colors import LightSource
ls = LightSource(azdeg=315, altdeg=45)
shadsurf = ls.shade(z, cmap=plt.cm.gray)
plt.imshow(shadsurf, origin='lower')
```
for an array of elevations `z` that has already been loaded. `azdeg=315` and `altdeg=45` are the default azimuth and elevation angles in degrees for the light source.

Make a shaded surface plot of Arthur’s Seat and experiment with different azimuth angles (0º to 360º) and elevation angles (0º to 90º) for the light source to see how the appearance of the plot changes. 

Contours can be added to a shaded map simply by putting `plt.contour()` after `plt.imshow()`. Pleasing effects can be produced by shading a coloured image map, but this requires making the shading semi-transparent; add keyword argument `alpha` with a value between 0 (fully transparent) and 1 (opaque).

## Geospatial data formats

In the exercise above, we have read our geospatial data from a text file, in exactly the same way we have been reading data from .txt and .csv files in previous weeks. These text files are known as [ASCII](https://www.pcmag.com/encyclopedia/term/ascii-file). They are easy to read with lots of different pieces of software, but can you see any limitations to them?

Where in the world is this topograhic map? How can you find the coordinate of any given point in it?

This file has no "metadata" to allow you to place it in the real world, or to give you other extra information to help you make correct use of it. There are file formats available that contain this metadata, known as ***self-describing files***, as they contain all the information you need to interpret all of the information contained. Some common formats for geospatial data are [geotiff](https://en.wikipedia.org/wiki/GeoTIFF), [HDF5](https://support.hdfgroup.org/HDF5/whatishdf5.html) and [netcdf](https://www.unidata.ucar.edu/software/netcdf/). All of these can contain some geospatial information, along with the metadata (stored in a header) needed. HDF5 and NetCDF can store very complex data. A geotiff is essentially the same as our ***"ArthursSeat.txt"***, but uses a more compact ***binary*** format, rather than storing them as number characters, and has the metadata header.

Some geotiff file have been proivided. These are lidar datasets collected by the Scottish government and provided for free from [here](https://remotesensingdata.gov.scot/data#/list). Some have been downloaded for you to the ***data*** directory.

Opening the data is a little more complicated than reading from a text file, as python loads it in to a ***structure***, which requires all the metadata to be set up correctly. But once you are used to the syntax and order, it is a very powerful tool.

The script below shows how to use the gdal library to geotiff a raster in to memory. Here the data is in a numpy array. Add to the end of this script to make the same plot you did for the text file DTM.

In [1]:
import gdal,osr
import numpy as np

# open a dataset object
filename='data/dtm/Coarsen.NT27SE_50CM_DTM_PHASE3.tif'
ds=gdal.Open(filename)

# read all geolocation information.
# This tells the computer how to map a 2D coordinate on to the Earth
proj=osr.SpatialReference(wkt=ds.GetProjection())
epsg=int(proj.GetAttrValue('AUTHORITY',1))

# read data dimensions from geotiff object
nX=ds.RasterXSize             # number of pixels in x direction
nY=ds.RasterYSize             # number of pixels in y direction

# geolocation tiepoint
transform_ds = ds.GetGeoTransform()# extract geolocation information
xOrigin=transform_ds[0]       # coordinate of x corner
yOrigin=transform_ds[3]       # coordinate of y corner
pixelWidth=transform_ds[1]    # resolution in x direction
pixelHeight=transform_ds[5]   # resolution in y direction

# read data. Returns as a 2D numpy array
data=ds.GetRasterBand(1).ReadAsArray(0,0,nX,nY)

# set no data values to 0, otherwise they will cause issues.
missingInd=np.where(data<-100.0)
if(len(missingInd)>0):
    data[missingInd[0],missingInd[1]]=0.0

# now plot the data contained in the 2D numpy array, data

ImportError: dlopen(/anaconda3/lib/python3.6/site-packages/osgeo/_gdal.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libpoppler.76.dylib
  Referenced from: /anaconda3/lib/libgdal.26.dylib
  Reason: image not found

We can also do quick maths on the spatial datasets. Here we have two ***raster*** layers (2D images), one representating the elevation of the ground (DTM) and the other the elevation of the highest object (DSM). If we subtract one from the other, we can get a map of height above ground. The code below defines a function to read in two filenames, one DTM and one DSM, and then calculate a new array of height above ground.

Add some code to that function to plot a graph of the height above ground.

In [ ]:
import gdal,osr
import numpy as np

def readTiff(filename):
    '''A function to read a geotiff'''
    ds=gdal.Open(filename)

    # geolocation tiepoint
    transform_ds = ds.GetGeoTransform()# extract geolocation information
    xOrigin=transform_ds[0]       # coordinate of x corner
    yOrigin=transform_ds[3]       # coordinate of y corner
    pixelWidth=transform_ds[1]    # resolution in x direction
    pixelHeight=transform_ds[5]   # resolution in y direction

    # read data. Returns as a 2D numpy array
    data=ds.GetRasterBand(1).ReadAsArray(0,0,nX,nY)
    
    # set no data values to 0, otherwise they will cause issues.
    missingInd=np.where(data<-100.0)
    if(len(missingInd)>0):
        data[missingInd[0],missingInd[1]]=0.0
        
    # pass data back
    return(data,xOrigin,yOrigin,pixelWidth,pixelHeight)


def heightAboveGround(dtmName,dsmName):
    '''
    A function to calculate height above ground from a DTM and DSM
    Note that this assumes the two datasets are aligned and the same resolution
    '''
    
    # open the DTM and DSM and read data
    tData,tX0,tY0,tXres,tYres=readTiff(dtmName)  # data plus all metadata
    sData,sX0,sY0,sXres,sYres=readTiff(dtmName)  # data plus all metadata
    
    # Subtract the two to get height, as two are aligned
    hData=sData-tData
    
    # pass back to the calling function
    return(hData,tX0,tY0,tXres,tYres)



# call those functions
dtmName='data/dtm/Coarsen.NT27SE_50CM_DTM_PHASE3.tif'
dsmName='data/dsm/Coarsen.NT27SE_50CM_DSM_PHASE3.tif'

# calculate the height array and metadata
hData,hX0,hY0,hXres,hYres=heightAboveGround(dtmName,dsmName)


## Batch processing

You could have done the processing above with a few clicks within QGIS. The real power of programming comes when you need to scale up the problem. You have 9 tiles covering the area from Musselburgh to a few km south. Using the functions above, write a loop to process all nine tiles and make a height figure for each.

The code below starts you off by creating a list of the DTM and DSM files.

In [1]:


dsmList=["data/dsm/Coarsen.NT26NE_50CM_DSM_PHASE3.tif","data/dsm/Coarsen.NT26SE_50CM_DSM_PHASE3.tif",
"data/dsm/Coarsen.NT27SE_50CM_DSM_PHASE3.tif","data/dsm/Coarsen.NT36NE_50CM_DSM_PHASE3.tif",
"data/dsm/Coarsen.NT36NW_50CM_DSM_PHASE3.tif","data/dsm/Coarsen.NT36SE_50CM_DSM_PHASE3.tif",
"data/dsm/Coarsen.NT36SW_50CM_DSM_PHASE3.tif","data/dsm/Coarsen.NT37SE_50CM_DSM_PHASE3.tif",
"data/dsm/Coarsen.NT37SW_50CM_DSM_PHASE3.tif"]

dtmList=["data/dtm/Coarsen.NT26NE_50CM_DTM_PHASE3.tif","data/dtm/Coarsen.NT26SE_50CM_DTM_PHASE3.tif",
"data/dtm/Coarsen.NT27SE_50CM_DTM_PHASE3.tif","data/dtm/Coarsen.NT36NE_50CM_DTM_PHASE3.tif",
"data/dtm/Coarsen.NT36NW_50CM_DTM_PHASE3.tif","data/dtm/Coarsen.NT36SE_50CM_DTM_PHASE3.tif",
"data/dtm/Coarsen.NT36SW_50CM_DTM_PHASE3.tif","data/dtm/Coarsen.NT37SE_50CM_DTM_PHASE3.tif",
"data/dtm/Coarsen.NT37SW_50CM_DTM_PHASE3.tif"]

print("DSM",dsmList[0])
print("DTM",dtmList[0])


# Using a loop and the functions in the code block above,
# create an image of a height map from each of the files in the list

DSM data/dsm/Coarsen.NT26NE_50CM_DSM_PHASE3.tif
DTM data/dtm/Coarsen.NT26NE_50CM_DTM_PHASE3.tif


## Additional exercises

File $\texttt{MtStHelens.txt}$ is a DEM of Mount St Helens on an 800×800 point grid with 20 m horizontal spacing. View it with the programs that you wrote for Arthur’s Seat, adjusting parameters as necessary.

### Plotting vectors

Files $\texttt{Psurf.txt}$, $\texttt{Uwind.txt}$ and $\texttt{Vwind.txt}$ contain surface pressure (hPa) and westerly and southerly components of the wind (m s$^{–1}$) over the North Atlantic on 1 September 2010. Make a single plot showing wind speed with colours and pressure with contours (isobars).

If you have named the arrays holding the westerly and southerly wind vector components `U` and `V`, wind vector arrows can be simply added to the plot by `plt.quiver(U,V)`. If you know a little meteorology, you will be able to check that the relationship between the isobars and the wind vectors makes sense.